In [9]:
#import pymilvus package
import random
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

Connects a server

In [2]:
connections.connect("default", host="localhost", port="19530")

Creates a collection

In [7]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8),
]
schema = CollectionSchema(
    fields, "hello_milvus is the simplest demo to introduce the APIs"
)
hello_milvus = Collection("hello_milvus", schema)


Inspect vectors in the collection

In [10]:
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()

Builds indexes on the entities

In [11]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)

Status(code=0, message=)

Loads the collection to memory and performs a vector similarity search:

In [12]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}
result = hello_milvus.search(
    vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"]
)


Performs a vector query:

In [13]:
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])

Performs a hybrid search:

In [14]:
result = hello_milvus.search(
    vectors_to_search,
    "embeddings",
    search_params,
    limit=3,
    expr="random > -12",
    output_fields=["random"],
)

print(result)


['["id: 2998, distance: 0.0, entity: {\'random\': -11.0}", "id: 425, distance: 0.22144004702568054, entity: {\'random\': -11.0}", "id: 2633, distance: 0.2515070140361786, entity: {\'random\': -11.0}"]', '["id: 2119, distance: 0.19571137428283691, entity: {\'random\': -11.0}", "id: 1317, distance: 0.3418614864349365, entity: {\'random\': -11.0}", "id: 1434, distance: 0.35981133580207825, entity: {\'random\': -11.0}"]']


Deletes entities by their primary keys:

In [15]:
expr = f"pk in [{entities[0][0]}, {entities[0][1]}]"
hello_milvus.delete(expr)

(insert count: 0, delete count: 2, upsert count: 0, timestamp: 445007161198379009, success count: 0, err count: 0)

Drops the collection:

In [ ]:
utility.drop_collection("hello_milvus")